In [33]:
import numpy as np
from functools import reduce
import torch
import operator

In [34]:
def create_data(shape, begin = 0, truncate = False, truncate_base = 13):
    size = reduce(lambda x, y: x * y, shape)
    res = []
    for i in range(size):
        res.append(i % 5)
    return np.array(res,dtype=np.float32).reshape(shape)

In [35]:
s13x3 = torch.nn.Conv2d(3, 12 , (1,1), (1,1),dilation = (1,1),padding = 0)

In [36]:
n = 1
h = 32
w = 32
ic = 3

In [37]:
def get_data(shape):
    size = reduce(operator.mul, shape, 1)
    data = []
    for i in range(size):
        data.append(i % 5)
    
    res = np.array(data).astype(np.float32).reshape(shape)
    return res

In [38]:
data = torch.from_numpy(get_data((n, ic, h, w)))

In [39]:
torch.onnx.export(s13x3,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/code/CodeReading/mnn/MNN/cmake-build-debug/conv1x1.onnx",   # where to save the model (can be a file or file-like object)
                   export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [40]:
import onnxruntime as ort

In [41]:
sess = ort.InferenceSession("/Users/bob/code/CodeReading/mnn/MNN/cmake-build-debug/conv1x1.onnx")

In [42]:
np_input = get_data((n, ic, h, w))

In [43]:
output = sess.run(None, {'data': np_input})[0]

In [44]:
output.flatten()[1]

-0.30818594

In [45]:
np_input.tofile("/Users/bob/code/CodeReading/mnn/MNN/cmake-build-debug/input.bin")

In [46]:
output.tofile("/Users/bob/code/CodeReading/mnn/MNN/cmake-build-debug/output.bin")